In [ ]:
!pip3 install --upgrade pip
!pip3 install mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-metadata to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 952.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from google.colab import drive


from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Load Model

drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/Dataset capstone/SIBI Baru/SIBI Dataset/'

Mounted at /content/drive


In [ ]:
# Mendapatkan daftar label dari dataset
print(model_path)
labels = []
for i in os.listdir(model_path):
  if os.path.isdir(os.path.join(model_path, i)):
    labels.append(i)
print("Labels:", labels)


/content/drive/MyDrive/Dataset capstone/SIBI Baru/SIBI Dataset/
Labels: ['none', 'P', 'S', 'Y', 'U', 'Q', 'X', 'T', 'V', 'R', 'W', 'I', 'E', 'L', 'N', 'O', 'F', 'G', 'M', 'H', 'K', 'B', 'D', 'A', 'C']


In [ ]:
# Define the number of examples to visualize
NUM_EXAMPLES = 10

# Visualize a few examples from each label
for label in labels:
    label_dir = os.path.join(model_path, label)
    example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
    fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(15, 2))
    for i, filename in enumerate(example_filenames):
        img = plt.imread(os.path.join(label_dir, filename))
        axs[i].imshow(img)
        axs[i].axis('off')
    fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')
    plt.show()


In [ ]:
# Load dataset using MediaPipe
data = gesture_recognizer.Dataset.from_folder(
    dirname= model_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

# Split dataset
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
# Set hyperparameters and options
hparams = gesture_recognizer.HParams(export_dir="exportedModel")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)

# Create model
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

In [ ]:
# Evaluate model
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss: {loss}, Test accuracy: {acc}")

In [ ]:
# Export model
model.export_model()

# List the exported model directory
!ls exportedModel

In [ ]:
import shutil
import os

# Path to the folder you want to download
folder_path = 'exportedModel'

# Zip file name
zip_file_name = 'exportedModel.zip'

# Create a zip file of the folder
shutil.make_archive(zip_file_name, 'zip', folder_path)

# Download the zip file
from google.colab import files
files.download(zip_file_name)